In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision.models as models

from PIL import Image
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import ImageEnhance
import itertools

from collections import Counter
import sys

In [2]:
import os
from google.colab import drive
drive.mount("/content/drive")
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/classify-leaves/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/classify-leaves/test.csv")

Mounted at /content/drive


read data

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train_datas_url=[]
train_datas_url = list(train.iloc[:, 0])
train_labels=[]
train_labels = list(train.iloc[:, 1])

images to tensor

In [ ]:
# 定义转换器
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.ColorJitter(contrast=0.5),  # 增强对比度
    transforms.Normalize(mean=[0.5], std=[0.5])  # 归一化
])

In [ ]:
train_images_tensor = []
for i in range(len(train_datas_url)):
    url = train_datas_url[i]
    image = Image.open(url).convert('L')
    tensor = transformer(image).type(torch.float16)
    train_images_tensor.append(tensor)

KeyboardInterrupt: ignored

labels to tensor

In [ ]:
# labels to nums
counter = Counter(train_labels)
labels_title = list(counter.keys())
labels_num = []
for i in range(len(labels_title)):
    labels_num.append(i)
# title - number

In [ ]:
# nums of output
print(len(labels_title))
print(len(labels_num))

176
176


In [ ]:
train_labels_tensor = []
for i in range(len(train_labels)):
    label_tensor = torch.tensor(labels_title.index(train_labels[i]))
    train_labels_tensor.append(label_tensor)

save and load tensor

In [ ]:
torch.save(train_images_tensor, 'train_images_tensor.pt')

In [2]:
train_images_tensor = torch.load('train_images_tensor.pt')

KeyboardInterrupt: 

In [ ]:
torch.save(train_labels_tensor, 'train_labels_tensor.pt')

In [ ]:
train_labels_tensor = torch.load('train_labels_tensor.pt')

In [ ]:
train_datas_tensor = torch.stack(train_images_tensor)
train_labels_tensor = torch.stack(train_labels_tensor)

In [ ]:
train_dataset = TensorDataset(train_labels_tensor, train_datas_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

model def

In [ ]:
# 加载预训练的VGG16模型
# 创建VGG16模型实例
net = torchvision.models.vgg16(pretrained=True)

# 修改输入层
net.features[0] = nn.Conv2d(1, 64, kernel_size=3, padding=1)

# 修改输出层
num_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(num_features, 176)

# 将模型移动到CUDA设备上
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device).half()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 174MB/s]


In [7]:
# 加载预训练的ResNet50模型
# 创建ResNet50模型实例
net = torchvision.models.resnet50(pretrained=True)

# 修改输入层
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# 修改输出层
num_features = net.fc.in_features
net.fc = nn.Linear(num_features, 176)

# 将模型移动到CUDA设备上
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device).half()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 127MB/s]


In [7]:
# 加载预训练的ResNet18模型
# 创建ResNet18模型实例
net = torchvision.models.resnet18(pretrained=True)

# 修改输入层
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# 修改输出层
num_features = net.fc.in_features
net.fc = nn.Linear(num_features, 176)

# 将模型移动到CUDA设备上
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device).half()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 127MB/s]


train && acc

In [10]:
del net, labels, inputs, outputs, loss, optimizer, criterion, images

NameError: ignored

In [11]:
torch.cuda.empty_cache()

In [10]:
for epoch in range(70):

    running_loss = 0.0
    num_images = 0
    for i, data in enumerate(train_dataloader, 0):
        labels, inputs = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        num_images += inputs.size(0)
        running_loss += loss.item()
        if num_images >= 14000: #18352
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, num_images, running_loss*256 / num_images))
            running_loss = 0.0
            break

print('Finished Training')

[1, 14080] loss: 0.412
[2, 14080] loss: 0.391
[3, 14080] loss: 0.378
[4, 14080] loss: 0.365
[5, 14080] loss: 0.346
[6, 14080] loss: 0.341
[7, 14080] loss: 0.327
[8, 14080] loss: 0.313
[9, 14080] loss: 0.308
[10, 14080] loss: 0.297
[11, 14080] loss: 0.285
[12, 14080] loss: 0.277
[13, 14080] loss: 0.266
[14, 14080] loss: 0.262
[15, 14080] loss: 0.255
[16, 14080] loss: 0.247
[17, 14080] loss: 0.240
[18, 14080] loss: 0.238
[19, 14080] loss: 0.226
[20, 14080] loss: 0.225
Finished Training


In [11]:
correct = 0
total = 0

# 计算要跳过的批次数量
skip_batches = int(14000/256) // train_dataloader.batch_size

# 使用itertools.islice()来跳过前  数据
data_iter = itertools.islice(train_dataloader, skip_batches, None)

with torch.no_grad():
    for data in data_iter:
        images, labels = data[1].to(device), data[0].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")


Accuracy on test set: 98.78%


In [ ]:
torch.save(net.state_dict(), 'resnet50_model_1.pth')

test